# Cross-Account vs Intra-Account Rules, and What is Root?

In particular, what does the following mean when used as the principal in an IAM resource policy?

"Principal": {"AWS": ["arn:aws:iam::111122223333:root]}

## Introduction

This lab examines the difference between IAM vs AWS Resource based policies. In particular, we seek to understand the policy evaluation logic for S3 buckets with cross account access. For a refresher on IAM basics, see [Reference Policies Evaluation Logic](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_evaluation-logic.html), which is valid for when the IAM Principal and S3 Resource are in the same AWS account.

To summarize the above, if an action is allowed by an identity-based policy, a resource-based policy, or both, then AWS allows the action. An explicit deny in either of these policies overrides the allow.

The situation changes for [cross account access](https://aws.amazon.com/premiumsupport/knowledge-center/cross-account-access-s3/). In this case, access must be explicitly allowed in both the principal's AWS access policy and the resource policy. Unfortunately, the latter reference does not mention the confused deputy issue for cross-account access which occurs when the trusted account is a 3rd party SaaS vendor. As a result, many vendors which operate on customer's S3 buckets do so insecurely.

For this lab, we will assume both AWS accounts are owned by the same entity and will leave confused deputy issues for Lab 4 - Direct Access vs Assume Role: Granting cross account access to resources.

<img src="s3-cross-account.png" class="left"/>

Granting permissions for Principal-A to access Resource-B when both are in the same account can be done by giving Principal-A a permissions policy to access Resource-B. Alternatively, cross-account access could be granted in a resource policy such as the following bucket policy.

The resources for this lab are S3 buckets we will refer to as mybucket1 and mybucket2 (actually mybucket-$random).
All roles shown below will have the same permissions policy which allows them to access generic S3 resources.

This lab requires an admin_A and admin_B which can create all the necessary resources. For the purposes of instruction
we will create resources as needed. If you are running this for students, the admin can run setupA.sh and setupB.sh. 

In [ ]:
import json

In [ ]:
%%bash --out bash_output
echo $RANDOM

In [ ]:
rand = str(bash_output.strip())
rand

In [ ]:
rand = str(123)

For this lab, you will need user credentials in two AWS accounts A and B and ~/.aws/credentials with blocks like
```
[profileA]
aws_access_key_id=AKIA**********
aws_secret_access_key=xxxxxxxxxxxxxxxx

[profileB]
aws_access_key_id=AKIA**********
aws_secret_access_key=xxxxxxxxxxxxxxxx
```

Fill in the values you wish to use for the RHS (right hand side) of profileA and profileB below. They must be the same as in your ~/.aws/credentials file.

In [ ]:
profileA = "profileA"
profileB = "profileB"
mybucket1 = "mybucket1-" + rand
mybucket2 = "mybucket2-" + rand
path = "/aws-labs/" # path acts as a prefix to IAM roles and policies. It can be used like tags to list our assets.

In [ ]:
!aws --profile $profileA sts get-caller-identity

In [ ]:
!aws --profile $profileB sts get-caller-identity

Use the output Arn's from above to fill in the values below. arn:aws:iam::...

In [ ]:
principalA_arn = "Here"
principalB_arn = "And here"
accountA = principalA_arn.split(':')[4]
accountB = principalB_arn.split(':')[4]
accountB

That works. However...

In [ ]:
%%bash
aws --profile "${profileA}" sts get-caller-identity
aws --profile "${profileB}" sts get-caller-identity
# The following evaluates $profileA to null
# aws --profile $profileA sts get-caller-identity
# aws --profile $profileB sts get-caller-identity

As you can see from above, we must take care as bash magic in cells may not behave as we expect. The output of the cell above should correspond to both profiles, but it does not. The moral is, try not to get fancy with bash in jupyter.


## Admin Setup

Create the 4 roles for the lab

In [ ]:
roleA1 = "roleA1-" + rand
roleA2 = "roleA2-" + rand
roleB1 = "roleB1-" + rand
roleB2 = "roleB2-" + rand

Create both assume policies

In [ ]:
# assume_A_policy.json
assume_A_policy = {
  "Version": "2012-10-17",
  "Statement": 
    {
      "Sid": "AssumeRolePolicyForS3ReaderRoleByExternal",
      "Effect": "Allow",
      "Principal": {"AWS": principalA_arn},
      "Action": "sts:AssumeRole"
    }
}

In [ ]:
# assume_B_policy.json
assume_B_policy = {
    "Version": "2012-10-17",
    "Statement": {
        "Sid": "AssumeRolePolicyB",
        "Effect": "Allow",
        "Principal": { "AWS": principalB_arn },
        "Action": "sts:AssumeRole"
    }
}

In [ ]:
assume_A_policy_str = "'" + json.dumps(assume_A_policy) + "'"
assume_B_policy_str = "'" + json.dumps(assume_B_policy) + "'"

Create the roles inside the accounts

In [ ]:
!aws --profile $profileA iam create-role --role-name $roleA1 \
                                        --path $path \
                                        --assume-role-policy-document $assume_A_policy_str

In [ ]:
!aws --profile $profileA iam create-role --role-name $roleA2 \
                                        --path $path \
                                        --assume-role-policy-document $assume_A_policy_str

In [ ]:
!aws --profile $profileB iam create-role --role-name $roleB1 \
                                        --path $path \
                                        --assume-role-policy-document $assume_B_policy_str

In [ ]:
!aws --profile $profileB iam create-role --role-name $roleB2 \
                                        --path $path \
                                        --assume-role-policy-document $assume_B_policy_str

Create the buckets for the lab. Note that unlike the IAM resources created, the bucket ARN does not include the account ID.

In [ ]:
!aws --profile $profileA s3api create-bucket --bucket $mybucket1
!aws --profile $profileA s3api create-bucket --bucket $mybucket2 \
                --create-bucket-configuration LocationConstraint=us-west-1

Copy some demo files inside each bucket

In [ ]:
!aws --profile $profileA s3 cp demo-vars.sh s3://$mybucket1/
!aws --profile $profileA s3 cp assume_A_policy.json s3://$mybucket2/

Check that the files were copied successfully, just in case

In [ ]:
!aws --profile $profileA s3 ls s3://$mybucket1
!aws --profile $profileA s3 ls s3://$mybucket2

Create a tagset and attach it to the bucket

In [ ]:
tagset = 'TagSet=[{Key=path,Value=' + path + '},{Key=rand,Value=' + rand + '}]'
tagset

In [ ]:
!aws --profile $profileA s3api put-bucket-tagging --bucket $mybucket1 \
                --tagging $tagset
!aws --profile $profileA s3api put-bucket-tagging --bucket $mybucket2 \
                --tagging $tagset

We have written a function, awsas, which is aws_run_as.sh
in this repo. It can be called as follows:

awsas --profile profile role [aws command args]

This does the magic of using user/service credentials in profileB to 
assume roleB1 and then run commands. Without this function, you have to get the response 
for assume-role and put them into environment variables or ~/.aws/credentials profile each time
as described [here](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-role.html). 
This is a very new beta script so drop into a bash shell if anything goes wrong.

Let's try it out!

In [ ]:
!awsas --profile $profileA $roleA1 sts get-caller-identity

This should return an assumed role with the name roleA1-$rand, the account it's in, and its ARN. If it doesn't, check your creds and privileges.

### Admin setup of managed policy

Define a function to format everything correctly

In [ ]:
def jdump(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, sort_keys=True, indent=4 * ' ')

Create the policy

In [ ]:
# permission-policy.json
iam_permission_policy_for_s3 = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "PermissionPolicyForS3Access",
      "Effect": "Allow",
      "Action":["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource": ["arn:aws:s3:::{}/*".format(mybucket1), 
                   "arn:aws:s3:::{}".format(mybucket1),
                  "arn:aws:s3:::{}/*".format(mybucket2), 
                   "arn:aws:s3:::{}".format(mybucket2)]
    }]
}
jdump(iam_permission_policy_for_s3 , "iam_permission_policy_for_s3.json")

Since we want to attach this policy to multiple roles, we will need to create a managed policy, 
rather than an inline policy which is embedded in the role.

In [ ]:
!aws --profile $profileA iam create-policy --policy-name iam_permission_policy_for_s3_$rand \
                                  --path "/aws-labs/" \
                                  --policy-document file://iam_permission_policy_for_s3.json

We must create the policy in each AWS account so repeat for B

In [ ]:
!aws --profile $profileB iam create-policy --policy-name iam_permission_policy_for_s3_$rand \
                                --path "/aws-labs/" \
                                --policy-document file://iam_permission_policy_for_s3.json

# Exercises

## 1. Confirm no access for roles with no permissions and default (no) bucket policy
So far, we have created policies and roles. When we create a role, we must include the assume-role 
trust policy which says who can assume the role. But so far it is just an empty container for us to 
put permissions policies in. Before we attach a bucket policy, let's check that we we can do with the roles
as they are.

In [ ]:
!awsas --profile $profileA $roleA1 s3 ls $mybucket1

Not really much, right? Let's see what happens next

## 2. Check that a role with no permissions can access a bucket with the right bucket policy

Let's create a bucket policy that allows roleA2 access to mybucket1

In [ ]:
# mybucket1_policy.json
mybucket1_policy = {
  "Version": "2012-10-17",
  "Statement": [{
      "Sid": "AssumeRolePolicyForS3ReaderRoleA",
      "Effect": "Allow",
      "Principal": {"AWS": ["arn:aws:iam::{}:root".format(accountA), 
                            "arn:aws:iam::{}:role/aws-labs/{}".format(accountA, roleA2)]},
      "Action": ["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource": ["arn:aws:s3:::{}/*".format(mybucket1), 
                   "arn:aws:s3:::{}".format(mybucket1)]

    }]
}
jdump(mybucket1_policy , "mybucket1_policy.json")
mybucket1_policy_str = "'"+json.dumps(mybucket1_policy).replace(' ', '')+"'"

Great, now lets attach the policy to mybucket1. Note that we still have no roles with permission policies attached.

In [ ]:
!aws --profile $profileA s3api put-bucket-policy --bucket $mybucket1 --policy $mybucket1_policy_str

Test if roleA1 can access it.

In [ ]:
!awsas --profile $profileA $roleA1 s3 ls s3://$mybucket1

Expect

An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

Let's try the same with roleA2 which is explicitly allowed in the bucket policy even though roleA2 has no permission policy attached yet.

In [ ]:
!awsas --profile $profileA $roleA2 s3 ls s3://$mybucket1

The timestamp doesn't matter, just the file name showing ListBucket worked.

### Conclusion
Allowing "root" on an S3 policy does not grant access to all principals in the account. However, explicitly allowing a role in the S3 policy permits access even if the role has no attached permissions. This is what we mean when we say "If the principal and the resource are in the same account, permission is the union of policies attached to the resource and principal."

## 3. Can a role with permissions access a bucket with no bucket policy?

Now let's attach the iam_permission_policy_for_s3 to roleA1. For this we need the policy arn. It's in our sourced variables, but could be obtained as follows:

In [ ]:
!aws --profile $profileA iam list-policies --path-prefix /aws-labs/ | jq -r '.Policies[].Arn'

And then pasted below:

In [ ]:
iam_permission_policy_for_s3_arn = "Here"

Attach the policy to the role

In [ ]:
!aws --profile $profileA iam attach-role-policy --role-name $roleA1 --policy-arn $iam_permission_policy_for_s3_arn

Next, let's verify that roleA1 can now access mybucket1

In [ ]:
!awsas --profile $profileA $roleA1 s3 ls s3://$mybucket1

Expect someting like

2020-05-03 13:57:39        185 demo-vars.sh

Again, the timestamp doesn't matter, just the filename showing that the bucket is accesible.

Now test that roleA1, which has an IAM permission policy that allows it to work with mybucket1 and mybucket2, does not require mybucket2 to have a bucket policy allowing it.
To do that, we simply try to list mybucket2's contents.

In [ ]:
!awsas --profile $profileA $roleA1 s3 ls s3://$mybucket2

Expect

2020-05-07 13:12:38        227 assume_A_policy.json

### Conclusion

We confirmed that an IAM policy attached to a role is all that is required to access a bucket, further supporting the "Union within an account" rule.

## 4. Can roleB1 with explicit IAM policy permission to access mybucket1 in accountA access it?

Now, we want to test if we can access mybucket1 in accountA by putting a specific role policy that allows us to do just that.

First, let's recall the current mybucket1 policy

In [ ]:
mybucket1_policy

Like it says there, it only gives accountA and roleA2 access to the bucket

Now, let's attach our existing iam_permission_policy_for_s3 policy to roleB1. For that, we need its ARN, which we can get like this:

In [ ]:
!aws --profile $profileB iam list-policies --path-prefix /aws-labs/ | jq -r '.Policies[].Arn'

Then, we paste it below

In [ ]:
iam_permission_policy_for_s3_arnB = "Here"

We attach it to the role

In [ ]:
!aws --profile $profileB iam attach-role-policy --role-name $roleB1 --policy-arn $iam_permission_policy_for_s3_arnB

And finally, we test if we can access the bucket

In [ ]:
!awsas --profile $profileB $roleB1 s3 ls s3://$mybucket1

Expect

An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

Since roleB1 is in a different account from mybucket1, both resource policy and IAM permission policy must explicitly grant access.

#### Now, let's try with mybucket2 in accountA after attaching a cross-account policy.

First, create the policy

In [ ]:
# mybucket2_policy.json
mybucket2_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:PutObject",
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Effect": "Allow",
            "Principal": {
                "AWS": [
                    "arn:aws:iam::{}:root".format(accountB),
                    "arn:aws:iam::{}:role/aws-labs/{}".format(accountB,roleB2)
                ]
            },
            "Resource": [
                "arn:aws:s3:::{}/*".format(mybucket2),
                "arn:aws:s3:::{}".format(mybucket2)
            ],
            "Sid": "BucketPolicyForS3MyBucket2"
        }
    ]
    
}
mybucket2_policy_str = "'"+json.dumps(mybucket2_policy)+"'"

Then put it in the bucket

In [ ]:
!aws --profile $profileA s3api put-bucket-policy --bucket $mybucket2 --policy $mybucket2_policy_str

And then try to list its contents from roleB1

In [ ]:
!awsas --profile $profileB $roleB1 s3 ls s3://$mybucket2

Expect success. This works because mybucket1 trusts the root of accountB which means that it trusts the admin of accountB to assign s3 permissions INCLUDING accessing external accounts.

What about roleB2 then? We haven't attached a permission policy to it yet, but it is explicitly allowed in mybucket2's resource policy.

In [ ]:
!awsas --profile $profileB $roleB2 s3 ls s3://$mybucket2

Expect an AccessDenied. Surprise! This result is not consistent with the result when the role and s3 bucket are in the same account. For cross-account, even when an s3 bucket explicitly names a resource from another account, the role must also have the permission attached.

### Conclusion

We couldn't access a bucket in another account without an explicit allow inside the bucket policy AND the IAM permissions policy, and that is perfectly fine, since if it let us, we could access any bucket in any account just by knowing the bucket's name.

## 5. Beware of conditional policies

Let's replace the existing mybucket2 policy with the following policy. It adds a condition block to only allow access to certain IPs.

In [ ]:
# mybucket2_conditional_policy.json
mybucket2_conditional_policy = {
  "Version":"2012-10-17",
  "Statement":[
    {
      "Sid":"AddCrossAccountPutPolicy",
      "Effect":"Allow",
      "Principal": {"AWS": ["arn:aws:iam::{}:root".format(accountB)]},
      "Action":["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource":["arn:aws:s3:::{}/*".format(mybucket2), "arn:aws:s3:::{}".format(mybucket2)],
      "Condition": {
        "IpAddress": {
          "aws:SourceIp": [
            "54.240.144.0/32",
            "54.240.144.0/24"
          ]
        }
        }
      }
    ]
}
mybucket2_conditional_policy_str = "'"+json.dumps(mybucket2_conditional_policy)+"'"

In [ ]:
!aws --profile $profileA s3api put-bucket-policy --bucket $mybucket2 --policy $mybucket2_conditional_policy_str

#### Question:
* Can we list bucket items from non-whitelisted IPs?

First, let's try to access the bucket from role B1, which we knew could access it before.

In [ ]:
!awsas --profile $profileB $roleB1 s3 ls s3://$mybucket2

Now, we can't access anymore, because our IP is not in the whitelisted options.

What about role A1?

In [ ]:
!awsas --profile $profileA $roleA1 s3 ls s3://$mybucket2

### Conclusion
RoleA1 can access the bucket because its IAM permission policy is not restricted and the decision is based on the union of what the IAM permission policy allows (yes) and what the s3 bucket policy allows (no). In contrast, for roleB1, the IAM permission policy (yes) intersected with the resource policy (no), leading to a denial of access.

# 6. The necessity of explicit Deny statements

What if you wish to only allow roleA1 and no other principal access to mybucket2? You might try to apply granular roles to all principals in accountA so that you never granted access to resource * for s3 operations to any principal. This is difficult to enforce. A better way is to apply an explicit Deny to all principals except roleA1 in the mybucket2 bucket policy.

For that, we need a new mybucket2 policy, which we'll create now

In [ ]:
mybucket2_deny_policy = {
    "Version": "2012-10-17",
    "Statement": [
      {
        "Sid": "ExplicitDenyBucketPolicyForAllBut",
        "Effect": "Deny",
        "Principal": {
          "AWS": ["arn:aws:iam::{}:role/aws-labs/{}".format(accountB,roleB2),
                  "arn:aws:iam::{}:role/aws-labs/{}".format(accountA,roleA2)]
         },
        "Action": ["s3:PutObject", "s3:GetObject", "s3:ListBucket"],
        "Resource": [ "arn:aws:s3:::{}/*".format(mybucket2),
                      "arn:aws:s3:::{}".format(mybucket2)
        ],
        "Condition": {"ArnNotLike": {
              "aws:SourceArn": ["arn:aws:iam::{}:role/aws-labs/{}".format(accountA,roleA2),
                                "arn:aws:iam::{}:role/aws-labs/{}".format(accountB,roleB2)]}
        }
      },
      {
        "Sid": "AllowCrossAccountForS3MyBucket2",
        "Effect": "Allow",
        "Principal": {
          "AWS": ["arn:aws:iam::{}:role/aws-labs/{}".format(accountB,roleB1)]
         },
        "Action": ["s3:PutObject", "s3:GetObject", "s3:ListBucket"],
        "Resource": [ "arn:aws:s3:::{}/*".format(mybucket2),
                      "arn:aws:s3:::{}".format(mybucket2)
        ]
      }
    ]
  }
mybucket2_deny_policy_str = "'"+json.dumps(mybucket2_deny_policy)+"'"

Then, we'll put it in the bucket, replacing the previous policy

In [ ]:
!aws --profile $profileA s3api put-bucket-policy --bucket $mybucket2 --policy $mybucket2_deny_policy_str

Now, let's try to read the bucket's contents from roleA1

In [ ]:
!awsas --profile $profileA $roleA1 s3 ls s3://$mybucket2

Great! Our policy lets us do that. Now let's try from roleA2

In [ ]:
!awsas --profile $profileA $roleA2 s3 ls s3://$mybucket2

So far so good, only roleA1 can access. Now, notice the second statement in the policy? It allows roleB1 from accountB access to the bucket. If that statement gets deleted, then only roleA1 can access the bucket. But let's try accessing it from roleB1 to see if it works

In [ ]:
!awsas --profile $profileB $roleB1 s3 ls s3://$mybucket2

Just in case, let's also try accessing the bucket from roleB2

In [ ]:
!awsas --profile $profileB $roleB2 s3 ls s3://$mybucket2

It's not letting us, just like we planned

### Conclusion
If we need to, we can create a bucket policy that denies permission to all principals, except the one we want, which is good for scalability and security best practices

# Congratulations! 

You've completed Lab2.

We'll sumarize the results as follows:

* When a role and resource are in the same account permssion is granted if either the role or resource grants access. This is called union.

* When a role and resource are in different accounts, permission must be granted by both the role and the resource. This is called intersection.

Execute the cell below if you want to delete everything from both accounts, to keep them as clean as possible, following security best practices. 

Warning: The s3 rb command will remove everything from both buckets and then delete them.

In [ ]:
!aws --profile $profileA iam detach-role-policy --role-name $roleA1 --policy-arn $iam_permission_policy_for_s3_arn
!aws --profile $profileB iam detach-role-policy --role-name $roleB1 --policy-arn $iam_permission_policy_for_s3_arnB
!aws --profile $profileA iam delete-policy --policy-arn $iam_permission_policy_for_s3_arn
!aws --profile $profileB iam delete-policy --policy-arn $iam_permission_policy_for_s3_arnB
!aws --profile $profileA iam delete-role --role-name $roleA1
!aws --profile $profileB iam delete-role --role-name $roleB1
!aws --profile $profileA iam delete-role --role-name $roleA2
!aws --profile $profileB iam delete-role --role-name $roleB2
!aws --profile $profileA s3 rb s3://$mybucket1 --force
!aws --profile $profileA s3 rb s3://$mybucket2 --force